In [ ]:
from autogen_ext.models.openai import AzureOpenAIChatCompletionClient
from dotenv import load_dotenv
load_dotenv()

import os

In [ ]:
from autogen_agentchat.agents import AssistantAgent, CodeExecutorAgent
from autogen_agentchat.ui import Console
from autogen_ext.code_executors.local import LocalCommandLineCodeExecutor
from autogen_agentchat.conditions import TextMentionTermination, MaxMessageTermination
from autogen_agentchat.teams import SelectorGroupChat

In [ ]:

model_client = AzureOpenAIChatCompletionClient(
    model="gpt-4o-mini",
    api_version="2024-11-20",
    azure_endpoint=os.getenv("OPENAI_API_HOST"),
    api_key=os.getenv("OPENAI_API_KEY"),
    )

code_assistant = AssistantAgent(
        name="code_assistant",
        system_message="You are a helpful assistant. Write all code in python and .sh whenever necessary. You are good at debugging code. Your job while debugging is to look at multiple possible options of solving the complex task at hand."
                        "You break down the task into smaller parts and solve them one by one. You are also good at writing code snippets and explaining them to the user."
                        "When asked to create the plot for any of the tasks always save it in a file with the extension .png and reply back with the file name."
                        "Reply only 'TERMINATE' if the task is done.",
        description="An agent that helps in debugging code and writing code snippets.",
        model_client=model_client,
    )

code_executor = CodeExecutorAgent(
    name="code_executor",
    code_executor=LocalCommandLineCodeExecutor(work_dir="coding"),
)

planning_agent = AssistantAgent(
    name="planning_agent",
    description="An agent for planning tasks, this agent should be the first to engage when given a new task.",
    model_client=model_client,
    system_message="""
    You are a planning agent.
    Your job is to break down complex tasks into smaller, manageable subtasks.
    Your team members are:
        code_assistant: Helps in debugging code and writing code snippets.
        code_executor: Executes the code snippets and debugging the code.

    You only plan and delegate tasks - you do not execute them yourself.

    When assigning tasks, use this format:
    1. <agent> : <task>

    After all tasks are complete, summarize the findings and end with "TERMINATE".
    """,
)

# The termination condition is a combination of text termination and max message termination, either of which will cause the chat to terminate.
termination = TextMentionTermination("TERMINATE") | MaxMessageTermination(10)


In [ ]:
planning_agent.description

In [ ]:
roles = {'planning_agent': planning_agent.description, 'code_assistant': code_assistant.description, 'code_executor': code_executor.description}
history = []
participants = ['planning_agent', 'code_assistant', 'code_executor']

In [ ]:
selector_prompt = """Select an agent to perform task.

{roles}

Current conversation context:
{history}

Read the above conversation, then select an agent from {participants} to perform the next task.
Make sure the planner agent has assigned tasks before other agents start working.
Only select one agent.
"""

text_mention_termination = TextMentionTermination("TERMINATE")
max_messages_termination = MaxMessageTermination(max_messages=25)
termination = text_mention_termination | max_messages_termination

In [ ]:

# The group chat will alternate between the assistant and the code executor.
group_chat = SelectorGroupChat(
    [planning_agent, code_assistant, code_executor], 
    termination_condition=termination,
    selector_prompt=selector_prompt,
    allow_repeated_speaker=True,
    
    model_client=model_client,)

In [ ]:
task = "Can you plot the stock price for AMD vs NVDA for the last 6 months? Also mark the date on the x-axis and the stock price on the y-axis. Save the plot as a .png file and reply back with the file name."


# `run_stream` returns an async generator to stream the intermediate messages.
stream = group_chat.run_stream(task=task)
# `Console` is a simple UI to display the stream.

# async for message in stream:
#     history.append(message)
#     print(message)

await Console(stream)
